In [12]:
!pip install matplotlib
!pip install seaborn
!pip install pandas
!pip install numpy
!pip install pyspark


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 

movies_path = "../data/movies_data.csv"

movies_df = pd.read_csv(
    movies_path,
    low_memory=False
)

conn = sqlite3.connect('../data/movie.db')

movies_df.to_sql('movies', conn, index=False, if_exists='replace')

result = pd.read_sql_query("select * from movies limit 5;", conn)
print(result)

conn.close()

      tconst titleType                   primaryTitle  isAdult  releaseYear  \
0  tt0000009     movie                     Miss Jerry        0         1894   
1  tt0000147     movie  The Corbett-Fitzsimmons Fight        0         1897   
2  tt0000574     movie    The Story of the Kelly Gang        0         1906   
3  tt0000591     movie               The Prodigal Son        0         1907   
4  tt0000615     movie             Robbery Under Arms        0         1907   

   releaseMonth releaseDate  runtime_consolidated  imdb_averageRating  \
0            10  1894-10-08                  45.0                 5.4   
1             1  1897-01-01                 100.0                 5.3   
2            12  1906-12-26                  70.0                 6.0   
3             6  1907-06-19                  90.0                 5.6   
4            10  1907-10-01                  60.0                 4.3   

   imdb_numVotes  ...  bayesian_averageRating  profit_ratio  \
0          218.0  ...  

In [14]:
print(movies_df.dtypes)

tconst                                object
titleType                             object
primaryTitle                          object
isAdult                                int64
releaseYear                            int64
releaseMonth                           int64
releaseDate                           object
runtime_consolidated                 float64
imdb_averageRating                   float64
imdb_numVotes                        float64
tmdb_averageRating                   float64
tmdb_numVotes                          int64
revenue                                int64
budget                                 int64
popularity                           float64
overview                              object
tagline                               object
genres                                object
production_companies                  object
keywords                              object
weighted_averageRating               float64
total_numVotes                       float64
bayesian_a

In [29]:
# Weighted Average Rating for IMDb Only where m = 5000
conn = sqlite3.connect('../data/movie.db')

imdb_query = """
with global_avg as (
    select
        avg(imdb_averageRating) as C
    from movies
),

qualified_movies as (
    select
        m.tconst,
        m.primaryTitle,
        m.imdb_averageRating,
        m.imdb_numVotes,
        g.C,
        5000 as m_value
    from movies m
    cross join global_avg g
    where 1=1
        and m.releaseYear = 2020
        and m.imdb_numVotes >= 5000 
),

weighted_ratings as (
    select
        primaryTitle,
        imdb_averageRating,
        imdb_numVotes,
        ((imdb_numVotes / (imdb_numVotes + m_value)) * imdb_averageRating + (m_value / (imdb_numVotes + m_value)) * C) as weighted_rating
    from qualified_movies
)

select
    primaryTitle,
    weighted_rating,
    imdb_averageRating, 
    imdb_numVotes
from weighted_ratings
order by 2 desc
limit 10
"""

imdb_result = pd.read_sql(imdb_query, conn)
print(imdb_result)

                                        primaryTitle  weighted_rating  \
0                                    Soorarai Pottru         8.599774   
1                                        Dil Bechara         8.220329   
2                                           Hamilton         8.213315   
3                                         The Father         8.149503   
4  Demon Slayer: Kimetsu no Yaiba - Mt. Natagumo Arc         8.101467   
5  Demon Slayer: Kimetsu no Yaiba - The Movie: Mu...         8.075118   
6                                  Dara of Jasenovac         7.982554   
7                                               Soul         7.976029   
8                                 My Octopus Teacher         7.957392   
9                                        Wolfwalkers         7.788587   

   imdb_averageRating  imdb_numVotes  
0                 8.7       126948.0  
1                 8.3       135886.0  
2                 8.3       124487.0  
3                 8.2       207380.0  
4

In [16]:
# Weighted Average Rating for TMDb Only
tmdb_query = """
with global_avg as (
    select
        avg(tmdb_averageRating) as C
    from movies
    where 1=1
        and tmdb_numVotes != 0
        and tmdb_averageRating != 0
),

qualified_movies as (
    select
        m.tconst,
        m.primaryTitle,
        m.tmdb_averageRating,
        cast(m.tmdb_numVotes as float) as tmdb_numVotes,
        g.C,
        5000 as m_value
    from movies m
    cross join global_avg g
    where 1=1
        and m.releaseYear = 2020
        and m.tmdb_numVotes >= 5000 
),

weighted_ratings as (
    select
        primaryTitle,
        tmdb_averageRating,
        tmdb_numVotes,
        ((tmdb_numVotes / (tmdb_numVotes + m_value)) * tmdb_averageRating + (m_value / (tmdb_numVotes + m_value)) * C) as weighted_rating
    from qualified_movies
)

select
    primaryTitle,
    weighted_rating,
    tmdb_averageRating,
    tmdb_numVotes
from weighted_ratings
order by 2 desc
limit 10
"""

tmdb_result = pd.read_sql(tmdb_query, conn)
print(tmdb_result)


                                        primaryTitle  weighted_rating  \
0                                               Soul         7.315832   
1                                             Onward         6.794690   
2                                 Sonic the Hedgehog         6.766055   
3                              A Quiet Place Part II         6.680590   
4                                              Tenet         6.661466   
5                                         Extraction         6.585468   
6                                  Bad Boys for Life         6.580694   
7                                           365 Days         6.568703   
8  Birds of Prey and the Fantabulous Emancipation...         6.568520   
9                                       Enola Holmes         6.557859   

   tmdb_averageRating  tmdb_numVotes  
0               8.150         9473.0  
1               7.726         5687.0  
2               7.345         8901.0  
3               7.511         5691.0  
4

In [17]:
# Weighted Average Rating for blended IMDb & TMDb
blended_query = """
with global_avg as (
    select
        avg(weighted_averageRating) as C
    from movies
    where 1=1
),

qualified_movies as (
    select
        m.tconst,
        m.primaryTitle,
        m.weighted_averageRating,
        m.total_numVotes,
        g.C,
        5000 as m_value
    from movies m
    cross join global_avg g
    where 1=1
        and m.releaseYear = 2020
        and m.total_numVotes >= 5000 
),

weighted_ratings as (
    select
        primaryTitle,
        weighted_averageRating,
        total_numVotes,
        ((total_numVotes / (total_numVotes + m_value)) * weighted_averageRating + (m_value / (total_numVotes + m_value)) * C) as weighted_rating
    from qualified_movies
)

select
    primaryTitle,
    weighted_rating,
    weighted_averageRating,
    total_numVotes
from weighted_ratings
order by 2 desc 
limit 10
"""

blended_result = pd.read_sql(blended_query, conn)
print(blended_result)

                                        primaryTitle  weighted_rating  \
0                                    Soorarai Pottru         8.599150   
1                                        Dil Bechara         8.219236   
2                                           Hamilton         8.213042   
3                                         The Father         8.149360   
4  Demon Slayer: Kimetsu no Yaiba - Mt. Natagumo Arc         8.098337   
5  Demon Slayer: Kimetsu no Yaiba - The Movie: Mu...         8.081725   
6                                  Dara of Jasenovac         7.981792   
7                                               Soul         7.979908   
8                                 My Octopus Teacher         7.956267   
9                                        Wolfwalkers         7.797755   

   weighted_averageRating  total_numVotes  
0                8.699597        127012.0  
1                8.299096        135969.0  
2                8.299177        125704.0  
3                8.1

In [18]:
# Checking for higheset aaa_index
aaa_query = """
    select 
        primaryTitle,
        aaa_index,
        weighted_averageRating,
        imdb_averageRating,
        tmdb_averageRating,
        total_numVotes,
        imdb_numVotes,
        tmdb_numVotes,
        popularity,
        profit_ratio,
        revenue,
        budget
    from movies
    order by 2 desc
    limit 10
"""

aaa_result = pd.read_sql(aaa_query, conn)
print(aaa_result)

                                        primaryTitle  aaa_index  \
0                           The Shawshank Redemption  10.000000   
1                                      The Godfather   9.908631   
2                                    The Dark Knight   9.890688   
3      The Lord of the Rings: The Return of the King   9.815040   
4                                       Pulp Fiction   9.792283   
5                                          Inception   9.782123   
6  The Lord of the Rings: The Fellowship of the Ring   9.776556   
7                                       Forrest Gump   9.762625   
8                                         Fight Club   9.762621   
9                                   Schindler's List   9.748585   

   weighted_averageRating  imdb_averageRating  tmdb_averageRating  \
0                9.295124                 9.3               8.702   
1                9.195639                 9.2               8.707   
2                8.995033                 9.0          

In [19]:
aaa_2020_query = """
    select
        primaryTitle,
        aaa_index,
        weighted_averageRating,
        imdb_averageRating,
        tmdb_averageRating,
        total_numVotes,
        imdb_numVotes,
        tmdb_numVotes,
        popularity,
        profit_ratio,
        revenue,
        budget
    from movies
    where 1=1
        and releaseYear = 2020
    order by 2 desc
    limit 10
"""

aaa_2020_result = pd.read_sql(aaa_2020_query, conn)
print(aaa_2020_result)

                                        primaryTitle  aaa_index  \
0                                         The Father   8.981133   
1                                              Tenet   8.832850   
2                                    Soorarai Pottru   8.810048   
3                                      Another Round   8.743324   
4  Demon Slayer: Kimetsu no Yaiba - The Movie: Mu...   8.728940   
5                                        Dil Bechara   8.646885   
6                                           Hamilton   8.625314   
7                              A Quiet Place Part II   8.610074   
8                                               Soul   8.574558   
9                                          Nomadland   8.526303   

   weighted_averageRating  imdb_averageRating  tmdb_averageRating  \
0                8.199365                 8.2               8.152   
1                7.298486                 7.3               7.191   
2                8.699597                 8.7          

In [31]:
conn = sqlite3.connect('../data/movie.db')
flops_query = """ 
    select
        releaseMonth,
        avg(popularity)
    from movies
    group by 1 
    order by 1
"""

flops_result = pd.read_sql(flops_query, conn)
print(flops_result)

    releaseMonth  avg(popularity)
0              1         1.747394
1              2         2.779084
2              3         2.912853
3              4         2.826271
4              5         3.064398
5              6         3.167482
6              7         3.692207
7              8         3.864428
8              9         3.555376
9             10         2.830692
10            11         2.630184
11            12         2.973902


In [21]:
conn.close()